## Youtube video link:
https://youtu.be/WqlMnxlAj5o?si=QilXRBz78v21AsUy

# Custom GPT Link:
https://chatgpt.com/g/g-ptgad0eJV-crewai-to-agency-swarm-converter

# Trip Planner Example
https://github.com/crewAIInc/crewAI-examples/tree/main/trip_planner

## Installation

In [ ]:
%%capture
!pip install git+https://github.com/VRSEN/agency-swarm.git

In [ ]:
%%capture
!pip install crewai==0.11.0 unstructured==0.10.25 pyowm==3.3.0 tools==0.1.9 python-dotenv==1.0.0

In [ ]:
import os
import getpass

# Set environment variables for API keys
os.environ['SERPER_API_KEY'] = getpass.getpass('Enter SERPER_API_KEY: ') # https://serper.dev/ (free tier)
os.environ['BROWSERLESS_API_KEY'] = getpass.getpass('Enter BROWSERLESS_API_KEY: ') # https://www.browserless.io/ (free tier)
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OPENAI_API_KEY: ')

## Step 1: Convert the Tools


## Browser tools

CrewAI

In [ ]:
import json
import os

import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html


class BrowserTools():
  @tool("Scrape website content")
  def scrape_and_summarize_website(website):
    """Useful to scrape and summarize a website content"""
    url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
    payload = json.dumps({"url": website})
    headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data=payload)
    elements = partition_html(text=response.text)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
      agent = Agent(
          role='Principal Researcher',
          goal=
          'Do amazing researches and summaries based on the content you are working with',
          backstory=
          "You're a Principal Researcher at a big company and you need to do a research about a given topic.",
          allow_delegation=False)
      task = Task(
          agent=agent,
          model="gpt-4o-mini",
          description=
          f'Analyze and summarize the content bellow, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}'
      )
      summary = task.execute()
      summaries.append(summary)
    return "\n\n".join(summaries)

Agency Swarm

In [ ]:
import json
import os
import requests
from agency_swarm.tools import BaseTool
from pydantic import Field
from unstructured.partition.html import partition_html
from agency_swarm import get_openai_client

class ScrapeAndSummarizeWebsite(BaseTool):
    """
    Tool for scraping and summarizing website content.
    """
    website: str = Field(..., description="The URL of the website to scrape and summarize.")

    def run(self):
        url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
        payload = json.dumps({"url": self.website})
        headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
        response = requests.request("POST", url, headers=headers, data=payload)
        elements = partition_html(text=response.text)
        content = "\n\n".join([str(el) for el in elements])
        content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
        summaries = []
        client = get_openai_client()

        for chunk in content:
            task_description = dedent(f"""
                Analyze and summarize the content below, make sure to include the most relevant information in the summary, return only the summary nothing else.

                CONTENT
                ----------
                {chunk}
            """)
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a Principal Researcher at a big company and you need to do research about a given topic."},
                    {"role": "user", "content": task_description}
                ],
            )
            summary = completion.choices[0].message.content
            summaries.append(summary)

        return "\n\n".join(summaries)


## Calculator Tools

CrewAI

In [ ]:
from langchain.tools import tool

class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"

Agency Swarm

In [ ]:
from agency_swarm.tools import BaseTool
from pydantic import Field

class CalculatorTool(BaseTool):
    """
    Tool for performing mathematical calculations such as sum, minus, multiplication, division, etc.
    """
    operation: str = Field(..., description="A mathematical expression to be evaluated, e.g., '200*7' or '5000/2*10'.")

    def run(self):
        try:
            result = eval(self.operation)
            return str(result)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"


## Search Tools

CrewAI

In [ ]:
import json
import os

import requests
from langchain.tools import tool


class SearchTools():

  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
      return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:top_result_to_return]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link: {result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next

      return '\n'.join(string)

Agency Swarm

In [ ]:
import json
import os
import requests
from agency_swarm.tools import BaseTool
from pydantic import Field

class SearchInternet(BaseTool):
    """
    Tool for searching the internet about a given topic and returning relevant results.
    """
    query: str = Field(..., description="The search query for the internet search.")

    def run(self):
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": self.query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        if 'organic' not in response.json():
            return "Sorry, I couldn't find anything about that, there could be an error with your Serper API key."
        else:
            results = response.json()['organic']
            string = []
            for result in results[:top_result_to_return]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}",
                        f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}",
                        "\n-----------------"
                    ]))
                except KeyError:
                    continue

            return '\n'.join(string)

# Step 2: Convert the Agents

Use the following prompt: "Convert this into agency swarm agents, using the existing API parameters:"

In [ ]:
from crewai import Agent
from langchain.llms import OpenAI

class TripAgentsCrew():
  def city_selection_agent(self):
    return Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        model="gpt-4o-mini",
        verbose=True)

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        model="gpt-4o-mini",
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
            CalculatorTools.calculate,
        ],
        model="gpt-4o-mini",
        verbose=True)

In [ ]:
from agency_swarm import Agent

class TripAgentsAgency:
    def city_selection_agent(self):
        return Agent(
            name="City Selection Expert",
            description="An expert in analyzing travel data to pick ideal destinations",
            instructions="Select the best city based on weather, season, and prices.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
            ],
            model="gpt-4o-mini",
        )

    def local_expert(self):
        return Agent(
            name="Local Expert at this city",
            description="A knowledgeable local guide with extensive information about the city, its attractions, and customs.",
            instructions="Provide the BEST insights about the selected city.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
            ],
            model="gpt-4o-mini",
        )

    def travel_concierge(self):
        return Agent(
            name="Amazing Travel Concierge",
            description="Specialist in travel planning and logistics with decades of experience.",
            instructions="Create the most amazing travel itineraries with budget and packing suggestions for the city.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
                CalculatorTool,
            ],
            model="gpt-4o-mini",
        )


# Step 3: Convert Tasks

Use the following prompt: "Convert these tasks into agency.get_completion methods. Pass the agent into recepient_agent parameter:"

In [ ]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasksCrew():

  def identify_task(self, agent, origin, cities, interests, range):
    return Task(description=dedent(f"""
        Analyze and select the best city for the trip based
        on specific criteria such as weather patterns, seasonal
        events, and travel costs. This task involves comparing
        multiple cities, considering factors like current weather
        conditions, upcoming cultural or seasonal events, and
        overall travel expenses.

        Your final answer must be a detailed
        report on the chosen city, and everything you found out
        about it, including the actual flight costs, weather
        forecast and attractions.
        {self.__tip_section()}

        Traveling from: {origin}
        City Options: {cities}
        Trip Date: {range}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def gather_task(self, agent, origin, interests, range):
    return Task(description=dedent(f"""
        As a local expert on this city you must compile an
        in-depth guide for someone traveling there and wanting
        to have THE BEST trip ever!
        Gather information about  key attractions, local customs,
        special events, and daily activity recommendations.
        Find the best spots to go to, the kind of place only a
        local would know.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, must-visit landmarks, weather forecasts, and
        high level costs.

        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.
        {self.__tip_section()}

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def plan_task(self, agent, origin, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 7-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        This itinerary should cover all aspects of the trip,
        from arrival to departure, integrating the city guide
        information with practical travel logistics.

        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special! {self.__tip_section()}

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def __tip_section(self):
    return "If you do your BEST WORK, I'll tip you $100!"

In [ ]:
from textwrap import dedent
from datetime import date

class TripTasksAgency:

    def identify_task(self, agency, agent, origin, cities, interests, date_range):
        task_description = dedent(f"""
            Analyze and select the best city for the trip based
            on specific criteria such as weather patterns, seasonal
            events, and travel costs. This task involves comparing
            multiple cities, considering factors like current weather
            conditions, upcoming cultural or seasonal events, and
            overall travel expenses.

            Your final answer must be a detailed
            report on the chosen city, and everything you found out
            about it, including the actual flight costs, weather
            forecast and attractions.
            {self.__tip_section()}

            Traveling from: {origin}
            City Options: {cities}
            Trip Date: {date_range}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def gather_task(self, agency, agent, origin, interests, date_range):
        task_description = dedent(f"""
            As a local expert on this city you must compile an
            in-depth guide for someone traveling there and wanting
            to have THE BEST trip ever!
            Gather information about key attractions, local customs,
            special events, and daily activity recommendations.
            Find the best spots to go to, the kind of place only a
            local would know.
            This guide should provide a thorough overview of what
            the city has to offer, including hidden gems, cultural
            hotspots, must-visit landmarks, weather forecasts, and
            high level costs.

            The final answer must be a comprehensive city guide,
            rich in cultural insights and practical tips,
            tailored to enhance the travel experience.
            {self.__tip_section()}

            Trip Date: {date_range}
            Traveling from: {origin}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def plan_task(self, agency, agent, origin, interests, date_range):
        task_description = dedent(f"""
            Expand this guide into a full 7-day travel
            itinerary with detailed per-day plans, including
            weather forecasts, places to eat, packing suggestions,
            and a budget breakdown.

            You MUST suggest actual places to visit, actual hotels
            to stay and actual restaurants to go to.

            This itinerary should cover all aspects of the trip,
            from arrival to departure, integrating the city guide
            information with practical travel logistics.

            Your final answer MUST be a complete expanded travel plan,
            formatted as markdown, encompassing a daily schedule,
            anticipated weather conditions, recommended clothing and
            items to pack, and a detailed budget, ensuring THE BEST
            TRIP EVER. Be specific and give it a reason why you picked
            each place, what makes them special! {self.__tip_section()}

            Trip Date: {date_range}
            Traveling from: {origin}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def __tip_section(self):
        return "If you do your BEST WORK, I'll tip you $100!"

# Step 4: Convert Crew

Use the following prompt: "Convert this into Agency:"

In [ ]:
%%time

from crewai import Crew, Agent
from textwrap import dedent

from dotenv import load_dotenv
load_dotenv()

class TripCrew:

  def __init__(self, origin, cities, date_range, interests):
    self.cities = cities
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgentsCrew()
    tasks = TripTasksCrew()

    city_selector_agent = agents.city_selection_agent()
    local_expert_agent = agents.local_expert()
    travel_concierge_agent = agents.travel_concierge()

    identify_task = tasks.identify_task(
      city_selector_agent,
      self.origin,
      self.cities,
      self.interests,
      self.date_range
    )
    gather_task = tasks.gather_task(
      local_expert_agent,
      self.origin,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      travel_concierge_agent,
      self.origin,
      self.interests,
      self.date_range
    )

    crew = Crew(
      agents=[
        city_selector_agent, local_expert_agent, travel_concierge_agent
      ],
      tasks=[identify_task, gather_task, plan_task],
      verbose=True
    )

    result = crew.kickoff()
    return result

print("## Welcome to Trip Planner Crew")
print('-------------------------------')
location = input(
  dedent("""
    From where will you be traveling from?
  """))
cities = input(
  dedent("""
    What are the cities options you are interested in visiting?
  """))
date_range = input(
  dedent("""
    What is the date range you are interested in traveling?
  """))
interests = input(
  dedent("""
    What are some of your high level interests and hobbies?
  """))

trip_crew = TripCrew(location, cities, date_range, interests)
crew_result = trip_crew.run()
print("\n\n########################")
print("## Here is you Trip Plan")
print("########################\n")
print(crew_result)

In [ ]:
%%time
from textwrap import dedent
from dotenv import load_dotenv
from agency_swarm import Agency, Agent

load_dotenv()

class TripAgency:

    def __init__(self, origin, cities, date_range, interests):
        self.cities = cities
        self.origin = origin
        self.interests = interests
        self.date_range = date_range

    def run(self):
        agents = TripAgentsAgency()
        tasks = TripTasksAgency()

        city_selector_agent = agents.city_selection_agent()
        local_expert_agent = agents.local_expert()
        travel_concierge_agent = agents.travel_concierge()

        agency = Agency(
            agency_chart=[
                city_selector_agent, local_expert_agent, travel_concierge_agent,
            ]
        )

        identify_task = tasks.identify_task(
            agency,
            city_selector_agent,
            self.origin,
            self.cities,
            self.interests,
            self.date_range
        )
        gather_task = tasks.gather_task(
            agency,
            local_expert_agent,
            self.origin,
            self.interests,
            self.date_range
        )
        plan_task = tasks.plan_task(
            agency,
            travel_concierge_agent,
            self.origin,
            self.interests,
            self.date_range
        )

        tasks_list = [identify_task, gather_task, plan_task]

        # Execute tasks in sequence
        results = []
        for task in tasks_list:
            result = task
            results.append(result)

        return plan_task


print("## Welcome to Trip Planner Agency")
print('-------------------------------')
location = input(
    dedent("""
        From where will you be traveling from?
    """))
cities = input(
    dedent("""
        What are the cities options you are interested in visiting?
    """))
date_range = input(
    dedent("""
        What is the date range you are interested in traveling?
    """))
interests = input(
    dedent("""
        What are some of your high level interests and hobbies?
    """))

trip_agency = TripAgency(location, cities, date_range, interests)
# Assuming `agency` is initialized elsewhere and passed here
agency_result = trip_agency.run()
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
print(agency_result)

# Compare the results

In [ ]:
from agency_swarm.messages import MessageOutput

In [ ]:
#@title Crew AI
MessageOutput("text", content=crew_result, sender_name="CrewAI Results", receiver_name="User").cprint()

In [ ]:
#@title Agency Swarm
MessageOutput("text", content=agency_result, sender_name="CrewAI Results", receiver_name="User").cprint()

In [ ]:
from agency_swarm import get_openai_client

client = get_openai_client()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Compare which travel plan is better. First provide your reasoning, then evaluation. Only output 1 option"},
        {"role": "user", "content": "CrewAI Travel Plan:" + crew_result + "\n\nAgency Swarm Travel Plan:" + agency_result},
    ],
    temperature=0,
)

print(completion.choices[0].message.content)